In [140]:
import csv
import pandas as pd
import numpy as np
import re
import os


In [141]:
kenya = pd.read_csv('KenyaClean.csv')
print(len(kenya.index))
mapping_df = pd.read_csv('kenya_narrativedata_mapping.csv')
print(len(mapping_df.index))

205
189


In [142]:
#mapping_df[['Q21.Income.Other', ]
mapping_df = mapping_df.rename(columns={'Q21.Income.Other': 'Q21.Income.Other.O6.Text'})

In [143]:
target_cols = ['Q21','Q93', 'Q28', 'Q95', 'Q97', 'Q99', 'Q147', 'Q148', 'Q150', 'Q162', 'Q163', 'Q165', 'Q166', 'Q170', 'Q171']

mapping_cols = [col for target in target_cols for col in mapping_df.columns if target in col and not '.FreeText' in col and not '.Clean' in col]
print(mapping_cols)

data_cols = [col for target in target_cols for col in kenya.columns if target in col and not '.FreeText' in col and not '.Clean' in col]
print(data_cols)

['Q21.Income.Other.O6.Text', 'Q21.Income.Other.Reponse1.Level1', 'Q21.Income.Other.Reponse1.Level2', 'Q21.Income.Other.Reponse2.Level1', 'Q21.Income.Other.Reponse2.Level2', 'Q93.BankAccountChallenges', 'Q28.FarmHealth.Text', 'Q95.BankLoanFacilityChallenges', 'Q95.BankLoanFacilityChallenges.Reponse1.Level1', 'Q95.BankLoanFacilityChallenges.Reponse1.Level2', 'Q95.BankLoanFacilityChallenges.Reponse2.Level1', 'Q97.BankCreditChallenges', 'Q99.SACCOChallenges', 'Q99.SACCOChallenges.Reponse1.Level1', 'Q99.SACCOChallenges.Reponse1.Level2', 'Q147.OtherInfoAreas', 'Q147.OtherInfoAreas.Reponse1.Level1', 'Q147.OtherInfoAreas.Reponse1.Level2', 'Q147.OtherInfoAreas.Reponse1.Level3', 'Q147.OtherInfoAreas.Reponse2.Level1', 'Q147.OtherInfoAreas.Reponse2.Level2', 'Q147.OtherInfoAreas.Reponse3.Level1', 'Q147.OtherInfoAreas.Reponse3.Level1.1', 'Q147.OtherInfoAreas.Reponse4.Level1', 'Q147.OtherInfoAreas.Reponse5.Level1', 'Q147.OtherInfoAreas.Reponse6.Level1', 'Q147.OtherInfoAreas.Reponse7.Level1', 'Q148.Id

In [152]:
def build_tag_set(row):
    clean_row = row.dropna()
    return list({item.lower().strip() for _, item in clean_row.iteritems() if item and item != '###'})

def map_categories(dataframe, mapping_table, join_key):
    mapping_table = mapping_table.fillna('###')
    mapped = pd.merge(dataframe, mapping_table, how='inner', on=[join_key])
    print(len(mapped.index))
    return mapped

def map_and_tag(dataframe, mapping_df, target_column, category_columns):    
    category_columns.append(target_column)
    mapped = map_categories(dataframe, mapping_df[category_columns], target_column)
    target_column_name = target_column + '.Tags'
    category_columns.remove(target_column)
    dataframe[target_column_name] = mapped[category_columns].apply(lambda row: build_tag_set(row),axis=1)
    print[dataframe[target_column_name]]

category_columns = ['Q21.Income.Other.Reponse1.Level1','Q21.Income.Other.Reponse1.Level2','Q21.Income.Other.Reponse2.Level1','Q21.Income.Other.Reponse2.Level2']
map_and_tag(kenya, mapping_df, 'Q21.Income.Other.O6.Text', category_columns)

category_columns = ['Q95.BankLoanFacilityChallenges.Reponse1.Level1', 'Q95.BankLoanFacilityChallenges.Reponse1.Level2', 'Q95.BankLoanFacilityChallenges.Reponse2.Level1']
map_and_tag(kenya, mapping_df, 'Q95.BankLoanFacilityChallenges', category_columns)

category_columns = ['Q99.SACCOChallenges.Reponse1.Level1', 'Q99.SACCOChallenges.Reponse1.Level2']
map_and_tag(kenya, mapping_df, 'Q99.SACCOChallenges', category_columns)

category_columns = ['Q147.OtherInfoAreas.Reponse1.Level1', 'Q147.OtherInfoAreas.Reponse1.Level2', 'Q147.OtherInfoAreas.Reponse1.Level3', 'Q147.OtherInfoAreas.Reponse2.Level1', 'Q147.OtherInfoAreas.Reponse2.Level2', 'Q147.OtherInfoAreas.Reponse3.Level1', 'Q147.OtherInfoAreas.Reponse3.Level1.1', 'Q147.OtherInfoAreas.Reponse4.Level1', 'Q147.OtherInfoAreas.Reponse5.Level1', 'Q147.OtherInfoAreas.Reponse6.Level1', 'Q147.OtherInfoAreas.Reponse7.Level1']
map_and_tag(kenya, mapping_df, 'Q147.OtherInfoAreas', category_columns)

category_columns = ['Q150.HelpToImplementChanges.Reponse1.Level1', 'Q150.HelpToImplementChanges.Reponse1.Level2', 'Q150.HelpToImplementChanges.Reponse2.Level1', 'Q150.HelpToImplementChanges.Reponse2.Level2', 'Q150.HelpToImplementChanges.Reponse3.Level1', 'Q150.HelpToImplementChanges.Reponse4.Level1']
map_and_tag(kenya, mapping_df, 'Q150.HelpToImplementChanges', category_columns)

category_columns = ['Q162.MainChallengesOnFarm.Reponse1.Level1', 'Q162.MainChallengesOnFarm.Reponse1.Level2', 'Q162.MainChallengesOnFarm.Reponse1.Level3', 'Q162.MainChallengesOnFarm.Reponse2.Level1', 'Q162.MainChallengesOnFarm.Reponse2.Level2', 'Q162.MainChallengesOnFarm.Reponse2.Level3', 'Q162.MainChallengesOnFarm.Reponse3.Level1', 'Q162.MainChallengesOnFarm.Reponse3.Level2', 'Q162.MainChallengesOnFarm.Reponse3.Level3', 'Q162.MainChallengesOnFarm.Reponse4.Level1', 'Q162.MainChallengesOnFarm.Reponse4.Level2',]
map_and_tag(kenya, mapping_df, 'Q162.MainChallengesOnFarm', category_columns)

category_columns = ['Q170.ChallengesOfSharingInfo.Reponse1.Level1', 'Q170.ChallengesOfSharingInfo.Reponse1.Level2', 'Q170.ChallengesOfSharingInfo.Reponse2.Level1']
map_and_tag(kenya, mapping_df, 'Q170.ChallengesOfSharingInfo', category_columns)

category_columns = ['Q171.MakeSharingEasier.Response1.Level1', 'Q171.MakeSharingEasier.Response1.Level2', 'Q171.MakeSharingEasier.Response2.Level1', 'Q171.MakeSharingEasier.Response2.Level2']
map_and_tag(kenya, mapping_df, 'Q171.MakeSharingEasier', category_columns)

68
[0                          [loans]
1                          [loans]
2                          [wages]
3                     [farm costs]
4                      [livestock]
5                     [farm costs]
6                     [farm costs]
7                     [farm costs]
8                     [farm costs]
9                     [farm costs]
10                    [farm costs]
11                    [farm costs]
12                    [farm costs]
13                    [farm costs]
14                    [farm costs]
15                         [wages]
16                    [farm costs]
17                    [farm costs]
18             [farm costs, wages]
19                         [wages]
20                         [wages]
21                         [wages]
22             [farm costs, wages]
23             [farm costs, wages]
24             [farm costs, wages]
25             [farm costs, wages]
26     [wages, savings, livestock]
27     [wages, savings, livestock]
28              

In [153]:

kenya.columns

Index([u'Unnamed: 0', u'SbjNum', u'Date', u'Srvyr', u'Duration', u'Upload',
       u'Latitude', u'Longitude', u'UTCDiff', u'VStart',
       ...
       u'Q184.GPS.Latitude', u'Q184.GPS.Longitude',
       u'Q21.Income.Other.O6.Text.Tags',
       u'Q95.BankLoanFacilityChallenges.Tags', u'Q99.SACCOChallenges.Tags',
       u'Q147.OtherInfoAreas.Tags', u'Q150.HelpToImplementChanges.Tags',
       u'Q162.MainChallengesOnFarm.Tags', u'Q170.ChallengesOfSharingInfo.Tags',
       u'Q171.MakeSharingEasier.Tags'],
      dtype='object', length=390)

In [146]:
kenya.to_csv('KenyaClean.mapped.csv')